# How do I make a new _project_?
### Overview
There are a number of API calls related to projects. Here we focus on _creating a **new**_ project.  We will show two options (A & B) of doing the same thing. Along the way, we will also show how to [list billing groups](http://docs.cancergenomicscloud.org/docs/list-your-billing-groups).

### Prerequisites
 1. You need your _authentication token_ and the API needs to know about it. See <a href="set_AUTH_TOKEN.ipynb">**set_AUTH_TOKEN.ipynb**</a> for details.
 2. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly here to prevent remaking the same project twice).
 
### Imports and Definitions
A single call is sufficient post a NewProject JSON and create a project. We will show two different options, both of which are defined in the apimethods.py file.

In [ ]:
from defs.apimethods import *

## (Option A) Use API() _object_
We start by listing all of your projects and your billing groups. Next we create the JSON that will be passed to the API to create the project. The dictionary should include:
* **billing_group** *Billing group* that will be charged for this project
* **description**   (optional) Project description
* **name**   Name of the project, may be *non-unique*<sup>1</sup>
* **tags**   List of tags, set tags[0] = 'tcga' if you will use *protected data*

**After** creating the project, you can re-check the project list and get *additional* details assigned by the CGC, including:

* **id**     _Unique_ identifier for the project, generated based on Project Name
* **href**   Address<sup>2</sup> of the project.
* **type**   (unimportant) this is always equal to 'v2'
* **flag**   (unimportant) this is set by the object constructor, here always 'longList':False 

<sup>1</sup> Please **don't** use non-unique *project names*. However, if you insist, the backend will allow it and assign a unique **id** to you project.

<sup>2</sup> This is the address where, by using API you can get this resource

In [ ]:
# [USER INPUT] Set project name here:
new_project_name = 'Life is Beautiful'

# LIST all projects
existing_projects = API('projects')                            
        
# What are my funding sources?
billing_groups = API('billing/groups')
# Pick the first group (arbitrary)
print((billing_groups.name[0] + \
       ' will be charged for computation and storage (if applicable) for your new project'))

# Set up the information for your new project
new_project = {
        'billing_group': billing_groups.id[0],
        'description': 'A project created by the API Cookbook.',
        'name': new_project_name, 
        'tags': ['tcga']
}
    
if new_project['name'] in existing_projects.name:
    # Your project (might) already exist
    print('A project with the same name already exists, please choose a unique name')
else:
    # CREATE the new project
    my_project = API(method='POST', data=new_project, path='projects')
    # (re)list all projects, to check that new project posted
    existing_projects = API(path='projects')
    # get ADDITONAL new project details 
    my_project = API(path=('projects/' + existing_projects.id[0])) 
    
    print('Your new project %s has been created.' % (my_project.name))
    if hasattr(my_project, 'description'): # need to check if description has been entered
        print('Project description: %s \n' % (my_project.description))
        
    if my_project.tags[0] == 'tcga':
        print('This is a PROTECTED data project.')
    else:
        print('This is an OPEN data project.')  

## (Option B) Use api_call() *function*
As above, we start by listing all your billing groups (but we don't check for existing projects<sup>3</sup>). Next we create the JSON that will be passed to the API to create the project. 

<sup>3</sup> Here we are **not** checking for existing projects with the same name, you can run this a few times to see how the API handles such requests.

In [ ]:
# [USER INPUT] Set project name here:
new_project_name = 'Keep on Smiling'

# what are my funding sources?
billing_groups = api_call(path='billing/groups')
# pick the first group (arbitrary)
print((billing_groups['items'][0]['name'] + \
       ' will be charged for computation and storage (if applicable) for your new project'))

# set up the information for your new project
new_project = {
        'billing_group': billing_groups['items'][0]['id'],
        'description': 'A project created by the API Cookbook.',
        'name': new_project_name,
        'tags': ['tcga']
}

# CREATE the new project
my_project = api_call(method='POST', data=new_project, path='projects')
# List most recent 100 projects, to check that new project posted
existing_projects = api_call(path='projects', method='GET', \
                            query={'limit':100, 'offset':0}, data=None)
# get ADDITONAL new project details 
my_project = api_call(method='GET', \
                     path=('projects/' + existing_projects['items'][0]['id']))

print(my_project)

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.cancergenomicscloud.org/docs/create-a-new-project)